In [2]:
import pandas as pd
import numpy as np


In [3]:
df=pd.read_csv('/Users/anto/Downloads/News Crashes.csv')

In [4]:
df.head()

,S. No.,Month,Crash Date,Crash Day,Article Date,Location,Million Plus City,State,LatLong,Vehicle 1,Vehicle/Object 2,Killed,Injured,Age,Gender,Road Type,Crash Type,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1.0,January,31-Dec-21,Friday,1-Jan-22,Alni,Nil,Maharashtra,"18.278457450374876, 76.00957388786227",Car,Tractor,4.0,0.0,"70,50,45,23",Both,NH,Head On Collision,NaN,NaN,NaN
1,2.0,January,31-Dec-21,Friday,1-Jan-22,Hariharganj,Nil,Jharkhand,"24.543681356839087, 84.28057070288547",Pick Up,Truck,6.0,18.0,NaN,Both,NH,Head On Collision,NaN,NaN,NaN
2,3.0,January,30-Dec-21,Thursday,1-Jan-22,Dabolim,Nil,Goa,"15.390068035586195, 73.85497360324335",Car,Divider,3.0,2.0,"27,24,24",Male,NH,Fixed Object Collision,NaN,NaN,NaN
3,4.0,January,30-Dec-21,Thursday,1-Jan-22,Triplicane,Chennai,Tamil Nadu,"13.067050304037062, 80.27014333317551",Pedestrian,Bus,1.0,0.0,50,Male,OR,Hit and Run,NaN,NaN,NaN
4,5.0,January,30-Dec-21,Thursday,1-Jan-22,PS Salai Junction,Chennai,Tamil Nadu,"13.039775780916695, 80.26465794801",Two Wheeler,Two Wheeler,1.0,1.0,NaN,Male,OR,Head On Collision,NaN,NaN,NaN


In [5]:
df.columns = df.columns.str.strip()

# Now try parsing again
df["Crash Date"] = pd.to_datetime(df["Crash Date"], format="%d-%b-%y", errors="coerce")

if "Article Date" in df.columns:
    df["Article Date"] = pd.to_datetime(df["Article Date"], format="%d-%b-%y", errors="coerce")
else:
    print("'Article Date' column not found — skipping...")

In [6]:
# Your dates look like "31-Dec-21" (day-month-year, 2-digit year)
df["Crash Date"] = pd.to_datetime(df["Crash Date"], format="%d-%b-%y", errors="coerce")
df["Article Date"] = pd.to_datetime(df["Article Date"], format="%d-%b-%y", errors="coerce")

# --- 2. Extract Latitude & Longitude from "LatLong" column ---
# "18.278457450374876, 76.00957388786227" → split by comma
df[["Latitude", "Longitude"]] = df["LatLong"].str.split(",", expand=True)

# Convert to numeric
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

# === Preview cleaned data ===
print(df[["Crash Date", "Article Date", "Latitude", "Longitude"]].head())

  Crash Date Article Date   Latitude  Longitude
0 2021-12-31   2022-01-01  18.278457  76.009574
1 2021-12-31   2022-01-01  24.543681  84.280571
2 2021-12-30   2022-01-01  15.390068  73.854974
3 2021-12-30   2022-01-01  13.067050  80.270143
4 2021-12-30   2022-01-01  13.039776  80.264658


In [7]:
import pandas as pd

# Example: load your data (adjust path as needed)
# df = pd.read_excel("your_file.xlsx")

# --- 1. Clean column names to snake_case ---
df.columns = (
    df.columns.str.strip()                 # remove leading/trailing spaces
              .str.lower()                 # lowercase
              .str.replace(" ", "_")       # replace spaces with _
              .str.replace(r"[^a-z0-9_]", "", regex=True)  # remove weird chars
)

print("Cleaned columns:", df.columns.tolist())

# --- 2. Parse dates ---
df["crash_date"] = pd.to_datetime(df["crash_date"], format="%d-%b-%y", errors="coerce")

if "article_date" in df.columns:
    df["article_date"] = pd.to_datetime(df["article_date"], format="%d-%b-%y", errors="coerce")

# --- 3. Extract latitude & longitude ---
if "latlong" in df.columns:
    df[["latitude", "longitude"]] = df["latlong"].str.split(",", expand=True)
    df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
    df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")

print(df.head())


Cleaned columns: ['s_no', 'month', 'crash_date', 'crash_day', 'article_date', 'location', 'million_plus_city', 'state', 'latlong', 'vehicle_1', 'vehicleobject_2', 'killed', 'injured', 'age', 'gender', 'road_type', 'crash_type', 'unnamed_17', 'unnamed_18', 'unnamed_19', 'latitude', 'longitude']
   s_no    month crash_date crash_day article_date           location  \
0   1.0  January 2021-12-31    Friday   2022-01-01               Alni   
1   2.0  January 2021-12-31    Friday   2022-01-01        Hariharganj   
2   3.0  January 2021-12-30  Thursday   2022-01-01            Dabolim   
3   4.0  January 2021-12-30  Thursday   2022-01-01         Triplicane   
4   5.0  January 2021-12-30  Thursday   2022-01-01  PS Salai Junction   

  million_plus_city        state                                latlong  \
0               Nil  Maharashtra  18.278457450374876, 76.00957388786227   
1               Nil    Jharkhand  24.543681356839087, 84.28057070288547   
2               Nil          Goa  15.3900

In [8]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from branca.element import MacroElement
from jinja2 import Template

# --- Prepare your dataframe (same as before) ---
df["crash_date"] = pd.to_datetime(df["crash_date"], errors="coerce")
df = df.dropna(subset=["latitude", "longitude"])

# --- Create Folium map centered on India ---
m = folium.Map(location=[22.5, 80], zoom_start=5, tiles="CartoDB positron")

# --- Add HeatMap with scaleRadius enabled ---
heat_data = df[["latitude", "longitude"]].dropna().values.tolist()
hm = HeatMap(
    heat_data,
    radius=25,          # base radius
    blur=30,
    min_opacity=0.6,
    max_zoom=20
)
m.add_child(hm)

# --- Inject JS to enable scaleRadius (radius scales with zoom) ---
class ScaleRadius(MacroElement):
    _template = Template("""
        {% macro script(this, kwargs) %}
            {{this._parent.get_name()}}.eachLayer(function(layer){
                if(layer.setOptions){
                    layer.setOptions({scaleRadius: true});
                }
            });
        {% endmacro %}
    """)
m.get_root().add_child(ScaleRadius())

# --- Save map ---
m.save("/Users/anto/Downloads/crash_map_scaled.html")
print("Interactive map saved with zoom-scaled radius")


Interactive map saved with zoom-scaled radius


In [12]:
import pandas as pd
import folium
from folium.plugins import HeatMap, HeatMapWithTime, Geocoder
from branca.element import MacroElement, Template

# --- Prepare dataframe ---
df["crash_date"] = pd.to_datetime(df["crash_date"], errors="coerce")
df = df.dropna(subset=["latitude", "longitude"])

# --- Create Folium map ---
m = folium.Map(location=[22.5, 80], zoom_start=5, tiles="CartoDB positron")

# === 1. Static HeatMap ===
heat_data = df[["latitude", "longitude"]].dropna().values.tolist()
hm = HeatMap(
    heat_data,
    radius=15,
    blur=12,
    min_opacity=0.3,
    max_opacity=0.6
)
m.add_child(hm)

# === 2. Time-enabled HeatMap ===
df["month_year"] = df["crash_date"].dt.to_period("M")
time_slices = []
for period, group in df.groupby("month_year"):
    time_slices.append(group[["latitude", "longitude"]].dropna().values.tolist())

hmt = HeatMapWithTime(
    time_slices,
    index=[str(p) for p in sorted(df["month_year"].dropna().unique())],
    radius=15,
    min_opacity=0.3,
    max_opacity=0.6,
    auto_play=True
)
m.add_child(hmt)

# === 2a. Force loop = true (JS injection) ===
loop_js = Template("""
{% macro script(this, kwargs) %}
var map = {{this._parent.get_name()}};
map.timeDimension.options.loop = true;
{% endmacro %}
""")
macro = MacroElement()
macro._template = loop_js
m.get_root().add_child(macro)

# === 3. Inject custom JS for zoom-dependent scaling ===
scale_js = Template("""
{% macro script(this, kwargs) %}
var map = {{this._parent.get_name()}};

function updateHeatmap(){
    var zoom = map.getZoom();
    var newRadius = 5 + zoom * 2;    // radius grows with zoom
    var newOpacity = Math.min(0.2 + zoom * 0.05, 0.9);  // opacity capped

    map.eachLayer(function(layer){
        if(layer.setOptions){
            layer.setOptions({
                radius: newRadius,
                maxOpacity: newOpacity,
                scaleRadius: true
            });
        }
    });
}

// Update on load + on zoom
map.on('zoomend', updateHeatmap);
updateHeatmap();
{% endmacro %}
""")
macro2 = MacroElement()
macro2._template = scale_js
m.get_root().add_child(macro2)

# === 4. Add Search / Geocoder ===
Geocoder(
    collapsed=False,              # keep box open
    add_marker=False,             # no pin, only zooms/pans
    position="topleft"
).add_to(m)

# --- Save map ---
m.save("/Users/anto/Downloads/crash_map_zoomscaled.html")
print("Interactive zoom-scaled heatmap with search saved: crash_map_zoomscaled.html")


Interactive zoom-scaled heatmap with search saved: crash_map_zoomscaled.html


In [13]:
import folium
from folium.plugins import HeatMap, Geocoder

# --- Create map ---
m_minors = folium.Map(location=[22.5, 80], zoom_start=5, tiles="CartoDB positron")

# --- Add heatmap for minor deaths only ---
heat_data = df_minors[["latitude", "longitude"]].dropna().values.tolist()
HeatMap(
    heat_data,
    radius=25,
    blur=15,
    min_opacity=0.4
).add_to(m_minors)

# --- Add search box / geocoder (like Google Maps search) ---
Geocoder(
    collapsed=False,              # keep box open
    add_marker=False,             # no pin, only zooms/pans
    position="topleft"
).add_to(m_minors)

# --- Save map ---
m_minors.save("/Users/anto/Downloads/minor_crashes.html")
print("Minor crash map saved with search box: minor_crashes.html")


Minor crash map saved with search box: minor_crashes.html
